### Import MLRun/Nuclio

In [1]:
import nuclio
from mlrun import code_to_function, new_function
import os
from os import path
import requests
import json

### Define Nuclio Function

In [2]:
#nuclio: start-code

In [3]:
import json
import random

def predict(context, data):
    """
    Emulates model prediction.
    """
    response = {"model" : context.model, "pred" : {}}
    for i, image in enumerate(data['data']):
        # pred = context.model.predict(d) -> Where you might perform model inferencing
        pred = random.choice(["cat", "dog"])
        response["pred"][image] = pred
    return context.Response(body=json.dumps(response))

def load_model():
    """
    Emulates loading model.
    """
    return "ML Model"

def init_context(context):
    """
    Initializes context - automatically called by Nuclio.
    
    Code you only need to run once such as initializing a model.
    """
    context.model = load_model()

def handler(context, event):
    """
    Main entrypoint into serverless function
    """
    return predict(context, json.loads(event.body))

In [4]:
#nuclio: end-code

### Build Custom Docker Image

In [5]:
# image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/custom-image"
image = "mlrun/mlrun"

In [6]:
# # Build Docker Image (only needs to be run once)
# build_image = new_function(name="build-image", kind="job")
# build_image.build_config(
#     image=image, base_image="mlrun/mlrun", commands=["pip install boto3"]
# )
# build_image.deploy(with_mlrun=False)

### Define Function Specs

In [7]:
fn = code_to_function(name='nuclio-fn', project="nuclio-template", kind='nuclio')
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.spec.base_spec['spec']['build']['baseImage'] = image

### Deploy Function

In [8]:
addr = fn.deploy()

> 2021-01-15 23:12:34,784 [info] Starting remote function deploy
2021-01-15 23:12:35  (info) Deploying function
2021-01-15 23:12:35  (info) Building
2021-01-15 23:12:35  (info) Staging files and preparing base images
2021-01-15 23:12:35  (info) Building processor image
2021-01-15 23:12:41  (info) Build complete
2021-01-15 23:12:45  (info) Function deploy complete
> 2021-01-15 23:12:46,419 [info] function deployed, address=3.17.244.7:30303


### Query Function

In [9]:
payload = {'data': ["image1", "image2", "image3"]}

response = requests.post(addr, json=json.dumps(payload))
predictions = response.json()

In [10]:
predictions

{'model': 'ML Model',
 'pred': {'image1': 'cat', 'image2': 'dog', 'image3': 'cat'}}